# 3.3 线性回归的简洁实现
- 使用深度学习框架（pytorch的nn模块）来简洁地实现线性回归模型。

## 3.3.1 生成数据集

In [1]:
import torch
from torch.utils import data
from d2l import torch as d2l

# 构造真实的w和b
true_w=torch.tensor([2,-3.4])
true_b=4.2

def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise.

    Defined in :numref:`sec_linear_scratch`"""
    X = d2l.normal(0, 1, (num_examples, len(w)))
    y = d2l.matmul(X, w) + b
    y += d2l.normal(0, 0.01, y.shape)
    return X, d2l.reshape(y, (-1, 1))

# 生成特征与标签
features,labels=synthetic_data(true_w,true_b,1000)

## 3.3.2 读取数据集
- 我们可以调用框架中现有的API来读取数据。

In [2]:
"""
我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。
"""
#@save
def load_array(data_arrays,batch_size,is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    # 随机返回样本数为batch_size的样本
    return data.DataLoader(dataset,
                           batch_size,
                           shuffle=is_train)

batch_size=10


data_iter=load_array((features,labels),batch_size) # 得到数为batch_size的样本

"""
为了验证是否正常工作，让我们读取并打印第一个小批量样本。
"""
# 使用iter构造Python迭代器，并使用next从迭代器中获取第一项。
next(iter(data_iter))


[tensor([[ 0.2521,  0.3670],
         [-2.8340, -0.6184],
         [ 0.6913,  0.1699],
         [-0.9767,  1.1784],
         [-0.9589, -0.1467],
         [ 0.2245, -0.4192],
         [-1.1645, -0.8282],
         [-0.2612, -2.3344],
         [ 0.5648,  0.4854],
         [ 1.3640, -0.4587]]),
 tensor([[ 3.4574],
         [ 0.6217],
         [ 5.0075],
         [-1.7586],
         [ 2.7781],
         [ 6.0839],
         [ 4.6821],
         [11.6149],
         [ 3.6981],
         [ 8.4857]])]

## 3.3.3 定义模型
- 对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。
- 我们首先定义一个模型变量net，它是一个Sequential类的实例
    - Sequential类将多个层串联在一起
    - 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推
- 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。
- 回顾上节的单层网络架构， 这一单层被称为全连接层（fully-connected layer）， 因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。

In [1]:
## nn是神经网络的缩写
from torch import nn

# 指定输入维度为2,输出为1的线性层（全连接层）
net = nn.Sequential(nn.Linear(2, 1))
net

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

## 3.3.4 初始化模型参数
- 深度学习框架通常有预定义的方法来初始化参数。


In [4]:
"""
在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。
"""
# 均值为0,方差为0.01的正态分布来对模型的w进行初始化
net[0].weight.data.normal_(0, 0.01)

# 偏差设为0
net[0].bias.data.fill_(0)


tensor([0.])

## 3.3.5 定义损失函数

In [5]:
"""
计算均方误差使用的是MSELoss类，也称为平方范数。 默认情况下，它返回所有样本损失的平均值
"""
loss = nn.MSELoss()

## 3.3.6 定义优化算法
- 小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。
- 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。

In [6]:
"""
小批量随机梯度下降只需要设置lr值，这里设置为0.03。
"""
# net.parameters()：模型的所有参数，包括w和b
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 3.3.7 训练
回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:
- 通过调用net(X)生成预测并计算损失l（前向传播）。
- 通过进行反向传播来计算梯度。
- 通过调用优化器来更新模型参数。

In [7]:
"""
为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。
"""
# 迭代数为3
num_epochs=3

for epoch in range(num_epochs):

    # 按照batch_size取数据
    for X,y in data_iter:
        l = loss(net(X),y)  # 将X与真实的y作loss
        trainer.zero_grad()  # 梯度清零
        l.backward()  # 反向传播，计算每个参数的梯度
        trainer.step()  # 根据梯度，更新模型的参数

    l=loss(net(features),labels)  # 扫描完所有数据之后，计算损失

    print(f'epoch{epoch+1}, loss{l:f}')


epoch1, loss0.000175
epoch2, loss0.000095
epoch3, loss0.000096


In [11]:
"""
下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。


正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。
"""
w=net[0].weight.data
print('w的估计误差：',true_w-w.reshape(true_w.shape))
b=net[0].bias.data
print('b的估计误差：',true_b-b)

w的估计误差： tensor([-0.0002, -0.0005])
b的估计误差： tensor([7.6771e-05])


## 3.3.8 小结
- 我们可以使用PyTorch的高级API更简洁地实现模型。

- 在PyTorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。

- 我们可以通过_结尾的方法将参数替换，从而初始化参数。